# Assign Random Groups In Canvas

In [ ]:
import os
import csv
from canvasapi import Canvas


In [ ]:
import collections
import random
import string

In [ ]:
students_per_group = 5
allow_uneven = True
group_category_name = 'TheGroupProject'

In [ ]:
token = MyPrivateKeys['Canvas']['token'] # Account->Settings->NewAccessToken

In [ ]:
MyPrivateKeys['Canvas']['site'], MyPrivateKeys['Canvas']['course_id']

In [ ]:
course = Canvas(MyPrivateKeys['Canvas']['site'], token).get_course(MyPrivateKeys['Canvas']['course_id'])

In [ ]:
# Get Students
enrollments = course.get_enrollments(state='active')
# Build a map of student ID to their name
id_to_name = dict( (e.user_id, e.user['name'])  for e in enrollments )

In [ ]:
id_by_role = collections.defaultdict(list)
for e in enrollments:
    if not e.user['name'].startswith("Test "):
        id_by_role[e.role].append((e.user_id,e.user['name']))
[(r,len(id_by_role[r])) for r in id_by_role]

In [ ]:
students = []
for e in enrollments:
    if e.role=='StudentEnrollment' and not e.user['name'].startswith("Test "):
        students.append((e.user_id,e.user['name']))
students[:3]

In [ ]:
if not allow_uneven:
    assert len(students) % students_per_group == 0

num_groups = int(len(students) / students_per_group)

if len(students) % students_per_group != 0:
    num_groups += 1

num_groups

In [ ]:
rand_students = list(students)
random.shuffle(rand_students)
rand_students[:3]

In [ ]:
assert len(group_assignments) <= 26

In [ ]:
# https://stackoverflow.com/questions/2130016/splitting-a-list-into-n-parts-of-approximately-equal-length
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))


In [ ]:
group_assignments = list(split(rand_students, num_groups))
group_assignments

### Assign Groups

In [ ]:
gcs = [_c for _c in course.get_group_categories() if _c.name == group_category_name]
assert len(gcs) == 1

gc = gcs[0]
assert len(list(gc.get_groups())) == 0

gc

In [ ]:
rev_let = reversed(list(string.ascii_uppercase))
to_iterate = list(zip(rev_let, group_assignments))
to_iterate

In [ ]:
gp = memb = 'off'

for lt, group_members in to_iterate:
    group_name = f"{group_category_name}_{lt}"
    print(group_name)
    gp = gc.create_group(name=group_name, description=f"B Boonstra Randomly Assigned Group {lt}")
    print(gp)
    for uid, uname in group_members:
        print(uname)
        memb = gp.create_membership(user=uid)
        print(memb)
    print('------')
    